##### Austin Hancock

## MSDS 7337 - Section 401
## Homework - #7
[Data Science @ Southern Methodist University](https://datascience.smu.edu/)

### Table of Contents
* [Description](#Description)
* [Tools](#Tools)
* [Question-1](#Question-1)
* [Question-2](#Question-2)
* [Question-3](#Question-3)

### <a name="Description"></a>Description
For the HW-7 assignment I will be addressing the following:

    - Using the reviews from HW 5, choose a clustering method and show at least two different ways the reviews can be clustered (changing k in k-means, where we "cut" in Agnes or Diana, etc.).
    - Explain what each cluster is generally centered on semantically.
    - Explain which clustering results from question 1 is preferable and why.

### <a name="Tools"></a>Tools

In [121]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import nltk; print("NLTK", nltk.__version__)
import bs4; print("bs4", bs4.__version__)
from bs4 import BeautifulSoup
import urllib
from urllib import request
import re; print("re", re.__version__)
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd; print("Pandas", pd.__version__)
import sklearn; print("sklearn", sklearn.__version__)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.cluster import KMeans
from collections import Counter

Windows-10-10.0.17134-SP0
Python 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
NLTK 3.2.4
bs4 4.6.0
re 2.2.1
Pandas 0.20.3
sklearn 0.19.1


### <a name="Question-1"></a>Question-1

Below, I am collecting the reviews from HW-5 and outputting them to a list called 'reviews'.

In [4]:
moana_url = "https://www.imdb.com/title/tt3521164/reviews?ref_=tt_ql_3"
frozen_url = 'https://www.imdb.com/title/tt2294629/reviews?ref_=tt_ql_3'
coco_url = 'https://www.imdb.com/title/tt2380307/reviews?ref_=tt_ql_3'
LionKing_url = 'https://www.imdb.com/title/tt0110357/reviews?ref_=tt_ql_3'

def find_permalinks(url):
    page = request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    review_containers = soup.find_all('div', class_ = 'review-container')
    for review in review_containers:
        permalinks.append("https://www.imdb.com" + review.find('a', attrs={'href': re.compile("/review")}).get('href'))
        
# Permalinks to reviews
permalinks = []

find_permalinks(moana_url)
find_permalinks(frozen_url)
find_permalinks(coco_url)
find_permalinks(LionKing_url)

# Grab the main review from each link and add to list
reviews = []

for link in permalinks:
    page = request.urlopen(link)
    soup = BeautifulSoup(page, 'html.parser')
    review_containers = soup.find_all('div', class_ = 'review-container')
    for review in review_containers:
        reviews.append(review.find(class_ = 'text show-more__control').text)

In [5]:
# Confirm extraction of reviews
print(reviews[0])

Moana is a return to the classic Disney formula, the clichés and characters ripped from a number of other animated films. However, the pure beauty and skill of the production rises the old story into new heights.Following from the success of Zootropolis, Moana follows a more traditional narrative we know and love; the princess who wishes for something more and is whisked on a supernatural adventure. We know this story so well yet Moana seems fresh and thrilling as if the plot was innovative. Perhaps this is due to the Polynesian setting or the morally ambiguous Maui, played perfectly by Dwayne Johnson, but most likely it is it the simple magic of Disney – the wonder for both children and adults has reached its peak with the perfection of the classic formula. For once, the clichés make the film more enjoyable. The quality of the animation helps too: it's clear they have reached the pinnacle of blending realistic textures with stylised designs, creating an aesthetic beauty that few other

Next, I tokenize the reviews and create a tfidf matrix from a count vectorizer.

In [147]:
# Tokenize
reviews_tokenized = []

for review in reviews:
    text = word_tokenize(review)
    text = [w.lower() for w in text]
    stop_words = set(stopwords.words('english'))
    filtered_sentence = []
    for w in text:
        if w not in stop_words:
            filtered_sentence.append(w)
        reviews_tokenized.append(filtered_sentence)

In [148]:
# Vectorize the reviews using CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer.fit_transform(reviews)

<100x3942 sparse matrix of type '<class 'numpy.int64'>'
	with 10880 stored elements in Compressed Sparse Row format>

In [149]:
# Print a list of word requencies
print(count_vectorizer.vocabulary_)

{'moana': 2270, 'return': 2930, 'classic': 604, 'disney': 979, 'formula': 1406, 'clichés': 613, 'characters': 559, 'ripped': 2961, 'number': 2408, 'animated': 188, 'films': 1328, 'pure': 2755, 'beauty': 336, 'skill': 3183, 'production': 2704, 'rises': 2962, 'old': 2437, 'story': 3327, 'new': 2372, 'heights': 1640, 'following': 1381, 'success': 3373, 'zootropolis': 3941, 'follows': 1382, 'traditional': 3587, 'narrative': 2349, 'know': 1940, 'love': 2093, 'princess': 2689, 'wishes': 3876, 'whisked': 3851, 'supernatural': 3399, 'adventure': 123, 'fresh': 1420, 'thrilling': 3531, 'plot': 2599, 'innovative': 1815, 'polynesian': 2620, 'setting': 3102, 'morally': 2298, 'ambiguous': 169, 'maui': 2175, 'played': 2590, 'perfectly': 2539, 'dwayne': 1062, 'johnson': 1888, 'likely': 2040, 'simple': 3160, 'magic': 2118, 'wonder': 3886, 'children': 578, 'adults': 121, 'reached': 2804, 'peak': 2527, 'perfection': 2538, 'make': 2128, 'film': 1326, 'enjoyable': 1142, 'quality': 2763, 'animation': 189, '

In [150]:
# Create tfidf matrix
tf_matrix = count_vectorizer.transform(reviews).toarray()
tfidfTran = TfidfTransformer(norm="l2")
tfidfTran.fit(tf_matrix)
tfidf_matrix = tfidfTran.transform(tf_matrix)
print(tfidf_matrix.shape)

(100, 3942)


In [151]:
# Return feature names from the vectorizer
feature_names = count_vectorizer.get_feature_names()
print(feature_names[100:120])

['actually', 'adam', 'adapt', 'adapted', 'adapting', 'adaption', 'add', 'added', 'adding', 'adds', 'adheres', 'adjust', 'admirable', 'admire', 'admit', 'adopt', 'adopted', 'adore', 'adored', 'adrian']


We now have a matrix and the features within our corpus. Using these, we can perform a K-Means clustering method.

The clustering method will cluster the reviews based on the TF-IDF features from the reviews.

In [152]:
# Define the k-means clustering function
def k_means(feature_matrix, num_clusters):
    km = KMeans(n_clusters=num_clusters, max_iter=10000)
    km.fit(feature_matrix)
    clusters = km.labels_
    return km, clusters

# Set number of clusters
num_clusters = 4

# Get clusters and assign the cluster labels to the reviews
km_obj, clusters = k_means(feature_matrix=tfidf_matrix, num_clusters=num_clusters)

Using this list, I next create a dataframe to see which reviews were assigned to which cluster.

In [154]:
# Create dataframe from the reviews list
reviews_df = pd.DataFrame(reviews)

# Add a new column for the movie title
reviews_df.insert(0, 'Movie', '')

# Since I pulled 25 reviews for each movie into an ordered list, I can assign the movie titles by index
reviews_df.iloc[0:25,0] = 'Moana'
reviews_df.iloc[25:50,0] = 'Frozen'
reviews_df.iloc[50:75,0] = 'Coco'
reviews_df.iloc[75:100,0] = 'LionKing'

# Rename columns to be 'Movie' and 'Review'
reviews_df.columns=['Movie', 'Review']

# Add clusters to dataframe
reviews_df['Cluster'] = clusters

In [166]:
print(reviews_df[:25])
print(reviews_df[25:50])
print(reviews_df[50:75])
print(reviews_df[75:])

    Movie                                             Review  Cluster
0   Moana  Moana is a return to the classic Disney formul...        0
1   Moana  Wow, what a great great movie! The visuals in ...        3
2   Moana  Tell me if you've seen this movie:Princess wan...        0
3   Moana  Instant classic.Easily ranks among the best fr...        0
4   Moana  2016 has been a particularly strong year for a...        0
5   Moana  From the opening lyrics to the final shot, Moa...        0
6   Moana  I have been largely unimpressed with Disney's ...        0
7   Moana  Henny's Movie Review: Moana (aka Vaiana)In thr...        0
8   Moana  I saw "Moana" tonight in the local movie theat...        2
9   Moana  So I watched Moana and boy, what a lazy movie ...        0
10  Moana  Personally, I thought this movie was boring, s...        0
11  Moana  A friend and I went with the kids to watch Moa...        0
12  Moana  This film is absolutely fantastic. Great CGI, ...        2
13  Moana  The posit

Let's see what happens if we use more clusters.

In [175]:
# Get clusters and assign the cluster labels to the reviews
km_obj_2, clusters_2 = k_means(feature_matrix=tfidf_matrix, num_clusters=7)

In [157]:
# Create dataframe from the reviews list
reviews_df_2 = pd.DataFrame(reviews)

# Add a new column for the movie title
reviews_df_2.insert(0, 'Movie', '')

# Since I pulled 25 reviews for each movie into an ordered list, I can assign the movie titles by index
reviews_df_2.iloc[0:25,0] = 'Moana'
reviews_df_2.iloc[25:50,0] = 'Frozen'
reviews_df_2.iloc[50:75,0] = 'Coco'
reviews_df_2.iloc[75:100,0] = 'LionKing'

# Rename columns to be 'Movie' and 'Review'
reviews_df_2.columns=['Movie', 'Review']

# Add clusters to dataframe
reviews_df_2['Cluster'] = clusters_2

In [173]:
print(reviews_df_2[:25])
print(reviews_df_2[25:50])
print(reviews_df_2[50:75])
print(reviews_df_2[75:])

    Movie                                             Review  Cluster
0   Moana  Moana is a return to the classic Disney formul...        0
1   Moana  Wow, what a great great movie! The visuals in ...        0
2   Moana  Tell me if you've seen this movie:Princess wan...        0
3   Moana  Instant classic.Easily ranks among the best fr...        4
4   Moana  2016 has been a particularly strong year for a...        4
5   Moana  From the opening lyrics to the final shot, Moa...        0
6   Moana  I have been largely unimpressed with Disney's ...        0
7   Moana  Henny's Movie Review: Moana (aka Vaiana)In thr...        0
8   Moana  I saw "Moana" tonight in the local movie theat...        0
9   Moana  So I watched Moana and boy, what a lazy movie ...        0
10  Moana  Personally, I thought this movie was boring, s...        0
11  Moana  A friend and I went with the kids to watch Moa...        0
12  Moana  This film is absolutely fantastic. Great CGI, ...        4
13  Moana  The posit

### <a name="Question-2"></a>Question-2

Now that I have clustered the reviews, I want to know a little bit more about why they clustered in the way that they did.

First, I want to see how many reviews were assigned to each cluster.

In [159]:
# Using the first K-Means model
c = Counter(clusters)
print(c.items())

dict_items([(0, 50), (3, 26), (2, 9), (1, 15)])


These results tell me that the number of reviews in each of the clusters are:

    - 0: 50
    - 1: 15
    - 2: 9
    - 3: 26
    
This is interesting, but still doesn't tell us what the features are in each cluster that determined its designation. In order to find out these features, we create the following function from 'Text Analytics with Python' by Dipanjan Sarkar.

In [160]:
def get_cluster_data(clustering_obj, review_data, feature_names, num_clusters, topn_features=10):
    # Create a dict to store the results
    cluster_details = {}
    
    # Get cluster centroids
    ordered_centroids = clustering_obj.cluster_centers_.argsort()[:, ::-1]
    
    # Get key features for each cluster
    # Get reviews belonging to each cluster
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster_num'] = cluster_num
        key_features = [feature_names[index] for index in ordered_centroids[cluster_num, :topn_features]]
        cluster_details[cluster_num]['key_features'] = key_features
        
        reviews = review_data[review_data['Cluster'] == cluster_num]['Review'].values.tolist()
        cluster_details[cluster_num]['Reviews'] = reviews
    
    return cluster_details

# To print the data from above in a clear format
def print_cluster_data(cluster_data):
    for cluster_num, cluster_details in cluster_data.items():
        print('Cluster {} details: '.format(cluster_num))
        print('-'*20)
        print('Key features: ', cluster_details['key_features'])
        print('Reviews in this cluster: ')
        print(', '.join(cluster_details['Reviews']))
        print('-'*40)

In [161]:
# Print cluster information
cluster_data = get_cluster_data(clustering_obj = km_obj, 
                                review_data=reviews_df, 
                                feature_names = feature_names, 
                                num_clusters=num_clusters,
                                topn_features = 5)

print_cluster_data(cluster_data)

Cluster 0 details: 
--------------------
Key features:  ['movie', 'disney', 'elsa', 'film', 'moana']
Reviews in this cluster: 
Moana is a return to the classic Disney formula, the clichés and characters ripped from a number of other animated films. However, the pure beauty and skill of the production rises the old story into new heights.Following from the success of Zootropolis, Moana follows a more traditional narrative we know and love; the princess who wishes for something more and is whisked on a supernatural adventure. We know this story so well yet Moana seems fresh and thrilling as if the plot was innovative. Perhaps this is due to the Polynesian setting or the morally ambiguous Maui, played perfectly by Dwayne Johnson, but most likely it is it the simple magic of Disney – the wonder for both children and adults has reached its peak with the perfection of the classic formula. For once, the clichés make the film more enjoyable. The quality of the animation helps too: it's clear t

Within the above output, we see that the top 5 features in each cluster are:

    - 0: 'movie', 'disney', 'elsa', 'film', 'moana'
    - 1: 'simba', ''king', 'lion', 'film', 'mufasa'
    - 2: 'great', 'moive', 'think', 'love', 'recommend'
    - 3: 'pixar', 'movie', 'coco', 'mexican', 'miguel'
    
Now, those are some interesting results!

From our clusters, we could see that Moan and Frozen reviews were typically assigned to cluster 0, Coco reviews assigned to cluster 3, and Lion King assigned to cluster 1, with cluster 2 designations sprinkled throughout. After looking at the features which were most important to each cluster, we begin to see why these reviews were given their designations.

For cluster 0, it is hard to determine why these two movies were clustered together. They both speak about disney, girls, and animation a lot, but it would have been nice to see a clean split between the reviews based on character names. Cluster 1 clearly identifies reviews of the movie The Lion King with high feature importance for two main characters. Cluster 2 could be categorized as postive reviews with high feature importance on 'great', 'love', and 'recommend'. The final cluster is mainly of the reviews for Coco with feature importance in 'pixar', 'coco', 'mexican', and 'miguel'.

To fine tune the model in the future, I could add some words to the stopwords list (movie, film, etc.) which provide no help to the clustering model as this is an exercise looking at movies.

### <a name="Question-3"></a>Question-3

Which clustering method is preferable?
   
    - Use models from question 1

In [174]:
# Print cluster information
cluster_data_2 = get_cluster_data(clustering_obj = km_obj_2, 
                                review_data=reviews_df_2, 
                                feature_names = feature_names, 
                                num_clusters=7,
                                topn_features = 5)

print_cluster_data(cluster_data_2)

Cluster 0 details: 
--------------------
Key features:  ['movie', 'moana', 'disney', 'movies', 'like']
Reviews in this cluster: 
Moana is a return to the classic Disney formula, the clichés and characters ripped from a number of other animated films. However, the pure beauty and skill of the production rises the old story into new heights.Following from the success of Zootropolis, Moana follows a more traditional narrative we know and love; the princess who wishes for something more and is whisked on a supernatural adventure. We know this story so well yet Moana seems fresh and thrilling as if the plot was innovative. Perhaps this is due to the Polynesian setting or the morally ambiguous Maui, played perfectly by Dwayne Johnson, but most likely it is it the simple magic of Disney – the wonder for both children and adults has reached its peak with the perfection of the classic formula. For once, the clichés make the film more enjoyable. The quality of the animation helps too: it's clear

When comparing the two clustering method, we can see that increasing the number of clusters can lead to a more granular view of the reviews. Below we see the top features for our clusters in the k=7 model

    - 0: ['movie', 'moana', 'disney', 'movies', 'like']
    - 1: ['elsa', 'anna', 'love', 'frozen', 'hans']
    - 2: ['movie', 'mexican', 'pixar', 'just', 'family']
    - 3: ['pixar', 'movie', 'toy', 'coco', 'inside']
    - 4: ['film', 'great', 'best', 'think', 'cast']
    - 5: ['simba', 'king', 'lion', 'scar', 'mufasa']
    - 6: ['coco', 'miguel', 'dead', 'land', 'family']
    
Unlike the k=4 model, this model was able to separate the movies into their own clusters. However, when it comes to movie reviews, we may not just want to identify which movie is being discussed. By using more clusters, we begin to gain more insights into what exactly it is about the movies that are being discussed. For instance, the reviews of the movie Coco were clustered into different groups, with some reviews talking more about Pixar and others talking more about family and the characters in the movie. If these deeper insights are what you are after, than increasing the number of clusters is one way of getting there.